In [6]:
!pip install trimesh

In [7]:

# importing sys
import sys
# adding src to the system path
sys.path.insert(0, './src')

import os
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from DataSetsLoader import DataSetsLoader
from PointNetFactory import PointNetFactory

In [8]:
# Global variables
NUM_POINTS = 2048
NUM_CLASSES = 10
NUM_POINT_AXIS = 3
BATCH_SIZE = 32

# Use hardware accelerator for training
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("GPUs Available: ", physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# Load Datasets
datasets_loader = DataSetsLoader(use_internet=True)
train_points, test_points, train_labels, test_labels, CLASS_MAP = datasets_loader.transform_to_tensorflow_dataset()

In [10]:
point_net_factory = PointNetFactory()

In [11]:

# Build the PointNet model
model = point_net_factory.create_model(NUM_POINTS, NUM_POINT_AXIS, NUM_CLASSES)
# model.summary()

In [12]:

# Train Model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
# Save Model
model.save_weights("./model.h5")

# Load Model
new_model = point_net_factory.create_model(NUM_POINTS, NUM_POINT_AXIS, NUM_CLASSES)
new_model.load_weights("./model.h5")

In [ ]:
# Predict Results
data = test_dataset.take(1)

points, labels = list(data)[0]
points = points[:10, ...]
labels = labels[:10, ...]

# run test data through model
preds = new_model.predict(points)
preds = tf.math.argmax(preds, -1)

points = points.numpy()

# plot points with predicted class and label
fig = plt.figure(figsize=(15, 10))
for i in range(10):
    ax = fig.add_subplot(2, 5, i + 1, projection="3d")
    ax.scatter(points[i, :, 0], points[i, :, 1], points[i, :, 2])
    ax.set_title(
        "pred: {:}, label: {:}".format(
            CLASS_MAP[preds[i].numpy()], CLASS_MAP[labels.numpy()[i]]
        )
    )
    ax.set_axis_off()
plt.show()

In [ ]:
dataset = test_dataset
total_count = 0
correct_count = 0

for element in dataset.as_numpy_iterator():
    points = element[0]
    labels = element[1]
    pred = model.predict(points)
    pred = tf.math.argmax(pred, -1)
    for index, point in enumerate(points):
        pred = model.predict(points)
        pred = tf.math.argmax(pred, -1)
        total_count += 1
        if labels[index] == pred[index]:
            correct_count += 1
            
accuracy  = correct_count / total_count
print("Accuracy  is {}".format(accuracy))